# 네이버 영화평 감성 분석

In [1]:
!pip install Konlpy > /dev/null 

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [4]:
# 네이버 영화 리뷰 데이터로 검색
train_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', sep='\t')
test_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', sep='\t')

In [5]:
print(train_df.shape, test_df.shape)
train_df.head(3)

(150000, 3) (50000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


### 1. 데이터 전처리
- 트레인 데이터셋

In [6]:
# Null 데이터가 있는지 확인
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [7]:
# Null 데이터 제거
train_df.dropna(how='any', inplace=True) 
train_df.shape

(149995, 3)

In [8]:
# 중복여부 확인
train_df.document.nunique()

146182

In [9]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True) 
train_df.shape

(146182, 3)

In [10]:
# 긍정(1)/부정(0) 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- 테스트 데이터셋

In [11]:
test_df.isna().sum()

id          0
document    3
label       0
dtype: int64

In [12]:
test_df.dropna(how='any', inplace=True)

In [13]:
test_df.document.nunique()

49157

In [14]:
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49157, 3)

In [15]:
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

### 2. 텍스트 전처리
- 트레인 데이터셋

In [16]:
# 한글 이외의 문자는 공백으로 처리하고 strip
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [17]:
# '' 만 남은 데이터 --> np.nan 으로 대체한 후 제거
train_df.document.replace('', np.nan, inplace=True)
train_df.document.isna().sum()

789

In [18]:
train_df.dropna(how='any', inplace=True)
train_df.shape

(145393, 3)

- 테스트 데이터셋

In [19]:
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ').str.strip()
test_df.document.replace('', np.nan, inplace=True)
test_df.dropna(how='any', inplace=True)
test_df.shape

(48852, 3)

- 전처리가 끝난 데이터는 저장

In [20]:
train_df.to_csv('naver_movie_train_전처리완료.tsv', sep='\t', index=False)
test_df.to_csv('naver_movie_test_전처리완료.tsv', sep='\t', index=False)

### 3. 한글 처리

In [23]:
from konlpy.tag import Okt
okt = Okt()

In [24]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
okt.morphs(text)

['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없다', '평점', '조정']

In [25]:
okt.morphs(text, stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [26]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','ㅋㅋ','ㅠㅠ','ㅎㅎ']
' '.join([word for word in okt.morphs(text, stem=True) if word not in stopwords])

'교도소 이야기 구먼 솔직하다 재미 없다 평점 조정'

In [27]:
from tqdm.notebook import tqdm

X_train = []
for sentence in tqdm(train_df.document):
    morphs = okt.morphs(sentence, stem=True)
    tmp_str = ' '.join([word for word in morphs if word not in stopwords])
    X_train.append(tmp_str)

  0%|          | 0/145393 [00:00<?, ?it/s]

In [28]:
X_test = []
for sentence in tqdm(test_df.document):
    morphs = okt.morphs(sentence, stem=True)
    tmp_str = ' '.join([word for word in morphs if word not in stopwords])
    X_test.append(tmp_str)

  0%|          | 0/48852 [00:00<?, ?it/s]

In [31]:
y_train = train_df.label.values 
y_test = test_df.label.values

### 4. 피쳐 변환 + 모델 학습/평가

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [32]:
cvect = CountVectorizer()
lr = LogisticRegression(random_state=2022)
pipeline = Pipeline([('CVECT', cvect), ('LR', lr)])
%time pipeline.fit(X_train, y_train)

CPU times: user 9.46 s, sys: 6.6 s, total: 16.1 s
Wall time: 9.84 s


Pipeline(steps=[('CVECT', CountVectorizer()),
                ('LR', LogisticRegression(random_state=2022))])

In [33]:
pipeline.score(X_test, y_test)

0.8273356259723246

### 5. 실제 데이터 테스트

In [34]:
review1 = '모든 국민이 봤으면 하는 영화입니다.'
review2 = '생각보다 지루하고 별로였네요... 보면서 좀 졸았습니다.'

In [35]:
# 전처리
import re
review1 = re.sub('[^가-힣]',' ', review1)
review2 = re.sub('[^가-힣]',' ', review2)

In [36]:
morphs = okt.morphs(review1, stem=True)
review1 = ' '.join([word for word in morphs if word not in stopwords])
morphs = okt.morphs(review2, stem=True)
review2 = ' '.join([word for word in morphs if word not in stopwords])

In [37]:
pipeline.predict([review1, review2])

array([1, 0])

### 6. 최적 파라메터 찾기

In [38]:
from sklearn.model_selection import GridSearchCV
params = {
    'CVECT__ngram_range': [(1,1),(1,2)],
    'CVECT__max_df': [0.95, 0.98],
    'LR__C': [1, 5]
}

In [39]:
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3, n_jobs=-1)
%time grid_pipe.fit(X_train, y_train)

CPU times: user 1min 1s, sys: 19.3 s, total: 1min 21s
Wall time: 4min 15s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('CVECT', CountVectorizer()),
                                       ('LR',
                                        LogisticRegression(random_state=2022))]),
             n_jobs=-1,
             param_grid={'CVECT__max_df': [0.95, 0.98],
                         'CVECT__ngram_range': [(1, 1), (1, 2)],
                         'LR__C': [1, 5]},
             scoring='accuracy')

In [40]:
grid_pipe.best_params_

{'CVECT__max_df': 0.95, 'CVECT__ngram_range': (1, 2), 'LR__C': 1}

In [41]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.8479898468844673